In [7]:
import evaluator_prob as evaluator
from qcg.generators import gen_supremacy, gen_hwea
import numpy as np
from scipy.stats import wasserstein_distance
from scipy.stats import chisquare

def cross_entropy(d1,d2):
    assert len(d1)==len(d2)
    h = 0
    for p,q in zip(d1,d2):
        if p==0:
            h += 0
        else:
            h += -p*np.log(q)
    return h

num_shots = int(1e5)
circ = gen_supremacy(4,4,8,order='75601234')

sv_noiseless_fc = evaluator.simulate_circ(circ=circ,backend='statevector_simulator',noisy=False,qasm_info=None)
qasm_info = [None,None,None,num_shots,None]
qasm_noiseless_fc = evaluator.simulate_circ(circ=circ,backend='qasm_simulator',noisy=False,qasm_info=qasm_info)

print('ce:')
print(cross_entropy(qasm_noiseless_fc,sv_noiseless_fc))
print(cross_entropy(sv_noiseless_fc,sv_noiseless_fc))
print('distance:')
print(wasserstein_distance(qasm_noiseless_fc,sv_noiseless_fc))
print(wasserstein_distance(sv_noiseless_fc,sv_noiseless_fc))
print('chi^2:')
print(chisquare(qasm_noiseless_fc,sv_noiseless_fc))
print(chisquare(sv_noiseless_fc,sv_noiseless_fc).statistic)

using statevector simulator
using noiseless qasm simulator 100000 shots
ce:
10.508830423887746
10.515123346528458
distance:
3.3340842463052196e-06
0.0
chi^2:
Power_divergenceResult(statistic=0.6463402517840394, pvalue=1.0)
Power_divergenceResult(statistic=0.0, pvalue=1.0)
